In [0]:

df = spark.read.parquet("/FileStore/ipl_final_cleaned.parquet")
df.show(5)
df.printSchema()

+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|year|        series_type|         series_name|match_no|match_type|          match_name|          match_href|         match_team1|         match_team2|match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|      MUMBAI INDIANS|ROYAL CHALLENGERS...| 2025-04-09 20:00:00|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...| CHENNAI SUPER KINGS|      DELHI CAPITALS| 2025-04-09 20:00:00|          null|       null|


In [0]:
from pyspark.sql.functions import col

completed_df = df.filter((col("match_team1").isNotNull()) & (col("match_team2").isNotNull()))
completed_df.show(5)


+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|year|        series_type|         series_name|match_no|match_type|          match_name|          match_href|         match_team1|         match_team2|match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|      MUMBAI INDIANS|ROYAL CHALLENGERS...| 2025-04-09 20:00:00|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...| CHENNAI SUPER KINGS|      DELHI CAPITALS| 2025-04-09 20:00:00|          null|       null|


In [0]:
columns_to_drop = [col for col in ["umpire1", "umpire2", "umpire3", "method", "dl_applied", "eliminator"] if col in completed_df.columns]
final_df = completed_df.drop(*columns_to_drop)


In [0]:
final_df.write.format("delta").mode("overwrite").save("/delta/ipl_table_data")


In [0]:
spark.sql("DROP TABLE IF EXISTS ipl_delta_table")

spark.sql("""
    CREATE TABLE ipl_delta_table
    USING DELTA
    LOCATION '/delta/ipl_table_data'
""")


Out[6]: DataFrame[]

In [0]:
dbutils.fs.ls("/FileStore/ipl_final_cleaned.parquet")



Out[8]: [FileInfo(path='dbfs:/FileStore/ipl_final_cleaned.parquet/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1744172255000),
 FileInfo(path='dbfs:/FileStore/ipl_final_cleaned.parquet/_committed_7441099480444998800', name='_committed_7441099480444998800', size=123, modificationTime=1744172255000),
 FileInfo(path='dbfs:/FileStore/ipl_final_cleaned.parquet/_started_7441099480444998800', name='_started_7441099480444998800', size=0, modificationTime=1744172255000),
 FileInfo(path='dbfs:/FileStore/ipl_final_cleaned.parquet/part-00000-tid-7441099480444998800-3762357d-42ce-4bb6-920a-cdd7c9624eef-71-1-c000.snappy.parquet', name='part-00000-tid-7441099480444998800-3762357d-42ce-4bb6-920a-cdd7c9624eef-71-1-c000.snappy.parquet', size=6692, modificationTime=1744172255000)]

In [0]:
df = spark.read.parquet("/FileStore/ipl_final_cleaned.parquet")
df.show(5)
df.printSchema()


+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|year|        series_type|         series_name|match_no|match_type|          match_name|          match_href|         match_team1|         match_team2|match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|      MUMBAI INDIANS|ROYAL CHALLENGERS...| 2025-04-09 20:00:00|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...| CHENNAI SUPER KINGS|      DELHI CAPITALS| 2025-04-09 20:00:00|          null|       null|


In [0]:
df.select("match_type").distinct().show(truncate=False)

+----------+
|match_type|
+----------+
|League    |
+----------+



In [0]:
completed_matches.printSchema()




root
 |-- year: integer (nullable = true)
 |-- series_type: timestamp (nullable = true)
 |-- series_name: string (nullable = true)
 |-- match_no: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_name: string (nullable = true)
 |-- match_href: string (nullable = true)
 |-- match_team1: string (nullable = true)
 |-- match_team2: string (nullable = true)
 |-- match_datetime_start: timestamp (nullable = true)
 |-- match_date_end: string (nullable = true)
 |-- match_venue: string (nullable = true)



In [0]:
from pyspark.sql.functions import split, col
fixed_matches = completed_matches.withColumn("match_team1", split(col("match_name"), " vs ")[0]) \
                                 .withColumn("match_team2", split(col("match_name"), " vs ")[1])

fixed_matches.select("match_team1", "match_team2").show(5)



+-----------+-----------+
|match_team1|match_team2|
+-----------+-----------+
+-----------+-----------+



In [0]:

df = spark.read.parquet("dbfs:/FileStore/ipl_cleaned.parquet/")
df.select("match_name").show(5, truncate=False)


+--------------------------------------------------------+
|match_name                                              |
+--------------------------------------------------------+
|MUMBAI INDIANS vs ROYAL CHALLENGERS BENGALURU, 1st Match|
|CHENNAI SUPER KINGS vs DELHI CAPITALS, 2nd Match        |
|SUNRISERS HYDERABAD vs KOLKATA KNIGHT RIDERS, 3rd Match |
|RAJASTHAN ROYALS vs PUNJAB KINGS, 4th Match             |
|KOLKATA KNIGHT RIDERS vs MUMBAI INDIANS, 5th Match      |
+--------------------------------------------------------+
only showing top 5 rows



In [0]:
df = spark.read.parquet("dbfs:/FileStore/ipl_cleaned.parquet/")
df.createOrReplaceTempView("ipl_data")
df.show(5, truncate=False)


+----+-------------------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------+---------------------------+--------------------+--------------+-----------+
|year|series_type        |series_name               |match_no|match_type|match_name                                              |match_href                                                                                   |match_team1          |match_team2                |match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------+---------------------------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|India

In [0]:
completed_matches = df.filter(df.match_name.isNotNull())
completed_matches.show(5, truncate=False)


+----+-------------------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------+---------------------------+--------------------+--------------+-----------+
|year|series_type        |series_name               |match_no|match_type|match_name                                              |match_href                                                                                   |match_team1          |match_team2                |match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------+---------------------------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|India

In [0]:
completed_matches.write.format("delta").mode("overwrite").saveAsTable("ipl_cleaned_delta")


In [0]:
spark.sql("SELECT * FROM ipl_cleaned_delta LIMIT 10").show(truncate=False)


+----+-------------------+--------------------------+--------+----------+----------------------------------------------------------------+----------------------------------------------------------------------------------------------+---------------------------+---------------------------+--------------------+--------------+-----------+
|year|series_type        |series_name               |match_no|match_type|match_name                                                      |match_href                                                                                    |match_team1                |match_team2                |match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------------+--------+----------+----------------------------------------------------------------+----------------------------------------------------------------------------------------------+---------------------------+---------------------------+--------------------+--------------

In [0]:
completed_matches.write.format("delta").mode("overwrite").saveAsTable("ipl_cleaned_delta")


In [0]:
%sql
SELECT * FROM ipl_cleaned_delta LIMIT 5;




year,series_type,series_name,match_no,match_type,match_name,match_href,match_team1,match_team2,match_datetime_start,match_date_end,match_venue
2021,2025-04-09T20:00:00.000+0000,Indian Premier League 2021,null,League,"MUMBAI INDIANS vs ROYAL CHALLENGERS BENGALURU, 1st Match",https://www.cricbuzz.com/cricket-scores/35612/mi-vs-rcb-1st-match-indian-premier-league-2021,MUMBAI INDIANS,ROYAL CHALLENGERS BENGALURU,2025-04-09T20:00:00.000+0000,null,null
2021,2025-04-09T20:00:00.000+0000,Indian Premier League 2021,null,League,"CHENNAI SUPER KINGS vs DELHI CAPITALS, 2nd Match",https://www.cricbuzz.com/cricket-scores/35617/csk-vs-dc-2nd-match-indian-premier-league-2021,CHENNAI SUPER KINGS,DELHI CAPITALS,2025-04-09T20:00:00.000+0000,null,null
2021,2025-04-09T20:00:00.000+0000,Indian Premier League 2021,null,League,"SUNRISERS HYDERABAD vs KOLKATA KNIGHT RIDERS, 3rd Match",https://www.cricbuzz.com/cricket-scores/35618/srh-vs-kkr-3rd-match-indian-premier-league-2021,SUNRISERS HYDERABAD,KOLKATA KNIGHT RIDERS,2025-04-09T20:00:00.000+0000,null,null
2021,2025-04-09T20:00:00.000+0000,Indian Premier League 2021,null,League,"RAJASTHAN ROYALS vs PUNJAB KINGS, 4th Match",https://www.cricbuzz.com/cricket-scores/35622/rr-vs-pbks-4th-match-indian-premier-league-2021,RAJASTHAN ROYALS,PUNJAB KINGS,2025-04-09T20:00:00.000+0000,null,null
2021,2025-04-09T20:00:00.000+0000,Indian Premier League 2021,null,League,"KOLKATA KNIGHT RIDERS vs MUMBAI INDIANS, 5th Match",https://www.cricbuzz.com/cricket-scores/35627/kkr-vs-mi-5th-match-indian-premier-league-2021,KOLKATA KNIGHT RIDERS,MUMBAI INDIANS,2025-04-09T20:00:00.000+0000,null,null
